In [241]:
"""
This program is developed as part of SDO/AIA's AIAPy Package @ LMSAL, Palo Alto, CA and performs: 
1. Image registration (rotation, translation, and scaling) of Level 1 AIA images 
2. Updates the header information
3. Walks you through the data science in a user-friendly manner via concise comments

Author: Dr. Manan Kocher, mkocher@lmsal.com
"""

#Import Statements------------------------------------------------------------------------------------------------------------------
import time
import numpy as np 
from inspect import signature 
import copy

import sunpy.map #this is the sub-module that holds \'Map\'
import sunpy.data.sample #this sub-module contains the sample data
from sunpy.map import Map #Map is a spatially-aware 2D array
from sunpy.map.sources.sdo import AIAMap, HMIMap

from sunpy.net import Fido, attrs as a #fido is an objext in sunpy.net - importing this here
                                       #Fido is a unified interface for searching and fetching
                                       #solar physics data irrespective of the underlying client 
                                       #or webservice through which the data is obtained
                                       #e.g. VSO, JSOC etc.

import astropy.units as u #handles defining/conversion of units
                          #e.g. lambda=171*u.angstrom
                          #lambda.value and lambda.unit can be printed
        
import drms #python package that can be used to access HMI/AIA/MDI data
import datetime
from datetime import timedelta #to support the subtraction and addition of time

import pandas as pd #pandas are

from sunpy.net import Fido, attrs as a
#--------------------------------------------------------------------------------------------------------------------------------
def aia_prep(aiamap,mpo_3Days=False,mpo_1Week=False,cutout=False): #aiamap is the only positional parameter passed into the code
                     #we can have additional keyword parameters to perform curated tasks for the user
    """
    This code performs image registration for one or more inputted AIAMaps, returning the updated AIAMaps as output
    
    Calling options (Copy-Paste into command line): 
    1. aia_prep(aiamap, mpo_3Days=True) #Full-disk image(s) provided, use latest pointing data updated every 3 days
    2. aia_prep(aiamap, mpo_1Week=True) #Full-disk image(s) provided, use pointing data updated weekly
    3. aia_prep(aiamap)                 #Full-disk image(s) provided, 3 day updated pointing data will be used
    4. aia_prep(aiamap, mpo_3Days=True, cutout=True)#cutout provided, use latest pointing data updated every 3 days
    5. aia_prep(aiamap, mpo_1Week=True, cutout=True)#cutout provided, use pointing data updated weekly
    6. aia_prep(aiamap, cutout=True) #cutout provided, 3 day updated pointing data will be used
    
    aiamap: Map object containing the 2D spatially-aware data array + metadata (index)
            Mandatory, if not passed the function will raise an error
    
    mpo_1Week: Would you like to use the 1 Week Master Pointing List? If yes, mark as True
    mpo_3Days: Would you like to use the 3 Day Master Pointing List? If yes, mark as True
    """
    print('This is the beta version of AIAPREP, written by Manan Kocher (mkocher@lmsal.com) for the AIAPy  package')
    
    t0=time.time() #starting the timer here
    
    #Step 1: Check to see if the passed file/files are compatible with the tasks aiaprep performs
    sig=signature(aia_prep) #contains the parameters and arguments passed into the function aia_prep
    params=sig.parameters  #writes out the parameters and their associated values here    
    #print(params) #print(len(params))
    if(len(params) < 1):    #making sure something is passed in which the function
        raise ValueError("Not enough parameters passed into the function, kindly provide atleast one AIAMap")
    if not isinstance(aiamap[0], (AIAMap, HMIMap)):
        raise ValueError("Input must be an AIAMap or HMIMap.")
    
    #Step 2: Iterating through each AIAMap & calling associated functions for aia_prep
    aiamap_copy=copy.copy(aiamap) #making a copy of aiamap, this will be edited
    n_img=np.size(aiamap_copy) #aiamap is a list
    print('Number of aiamaps are {0}'.format(n_img))
    maps_updated=[]
    for i in np.arange(0,n_img):
        updated_header_jsoc=sdo_master_pointing(aiamap_copy[i],mpo_1Week, mpo_3Days, cutout)#calling function to get pointing info
        print('-----------------------------------------')
        print('The information that needs to be updated in the meta data for aiamap # {0} are listed below:'.format(i))
        print(updated_header_jsoc) #this will return only one set of pointing pars (but in list form), since one aiamap was sent
        print('-----------------------------------------')
        map_updated1=header_update(aiamap_copy[i],updated_header_jsoc,mpo_1Week,mpo_3Days,cutout) #calling function that will update the header file
        #print(map_updated1[0].meta)
        maps_updated.append(map_updated1)
    return(map_updated1)
    
#----------------------------------------------------------------------------------------------------------------------
def header_update(aiamap,updated_header_jsoc,mpo_1Week=False,mpo_3Days=False,cutout=False): #this used to be called aia2wcsmin():
    
    """
    The header information of the AIAMap are updated using the latest JSOC Pointing Information, concluding the 
    first step of AIAMap update from Level 1 to 1.5.
    Inputs of 1. aiamap(s) 2. the updated header information (record number, sun center coordinates, 
    instrument rotation, image scale in arcsec/pixel of CCD) from sdo_master_pointing.py, and 3. whether image
    is full disk or cutout.
    
    Calling options (Copy-Paste into command line): 
    1. header_update(aiamap,updated_header_jsoc) #Full-disk image(s) provided along with jsoc pointing data
    2. header_update(aiamap,updated_header_jsoc,cutout=False) #Full-disk image(s) provided along with jsoc pointing data
    3. header_update(aiamap,updated_header_jsoc,cutout=True) #cutout image(s) provided along with jsoc pointing data    
    """
    print('Header Update Begins Now')
    #--------------------------------------------------------------------------------------------------------------------    
    #Step 1: Ensuring the AIAMap(s) input is a list, when more than one AIAMap is passed it will be a list, but 
             #if only 1 AIAMap is passed, it will have to be converted to a list of 1. 

    aiamap_list=[]
    aiamap_copy=copy.deepcopy(aiamap)
    if isinstance(aiamap_copy,__builtins__.list)!=True: #in order to loop through one or more AIAMaps, they have to be in list form
        aiamap_list.append(aiamap_copy) #if it's not a list, make it a list (only happens when it's just 1 AIAMap)
    if isinstance(aiamap_copy,__builtins__.list)==True:
        aiamap_list=aiamap_copy 
    #--------------------------------------------------------------------------------------------------------------------    
    #Step 2: Checking Inputs
    sig=signature(header_update) #contains the parameters and arguments passed into the function header_update
    params=sig.parameters  #writes out the parameters and their associated values here    
    #print(params) #print(len(params))
    if(len(params) < 2):    #making sure aiamap and jsoc pointing data are passed
        raise ValueError("Not enough parameters passed into the function, kindly provide atleast one AIAMap and the JSOC Pointing Info file")
    if not isinstance(aiamap_list[0], (AIAMap, HMIMap)):
        raise ValueError("Input must be an AIAMap or HMIMap.")
    #The code will automatically raise an error if the Pointing data isn't produced form sdo_master_pointing    
    
    if(len(aiamap_list)!=len(updated_header_jsoc)):
        raise ValueError("The number of AIAMaps and corresponding JSOC Master Pointing lists donot match")
    
    n_img=len(aiamap_list) #number of AIAMaps     
    for f in np.arange(0,n_img):       
        #--------------------------------------------------------------------------------------------------------------------    
        #Step 3: Using AIA Map Information used to construct custom JSOC Keyword names        
        wavelnth=aiamap_list[f].wavelength #astropy.units.quantity.Quantity, value+units
        wavelnth_num=wavelnth.value #Just the numerical portion of the wavelength
        x0_pointing='A_'+str(int(wavelnth_num))+'_X0'
        y0_pointing='A_'+str(int(wavelnth_num))+'_Y0'
        instrot_pointing='A_'+str(int(wavelnth_num))+'_INSTROT'
        imscale_pointing='A_'+str(int(wavelnth_num))+'_IMSCALE'
        recnum='*recnum*'
    
        #Step 4: updating the header data of the aiamap passed to this code
        aiamap_list[f].meta['lvl_num']=1.5
        #aiamap[0].meta['mpo_rec']
        aiamap_list[f].meta['inst_rot']=updated_header_jsoc[f][instrot_pointing].values
        aiamap_list[f].meta['imscl_mp']=updated_header_jsoc[f][imscale_pointing].values
        aiamap_list[f].meta['x0_mp']=updated_header_jsoc[f][x0_pointing].values
        aiamap_list[f].meta['y0_mp']=updated_header_jsoc[f][y0_pointing].values
    
        #Step 5: Updating the MPO Record Number based on the MPO requests by the user
        if mpo_1Week == True:
            aiamap_list[f].meta['mpo_rec']='MPO_1Week_#'+str(updated_header_jsoc[f][recnum].values)
        if mpo_3Days == True:
            aiamap_list[f].meta['mpo_rec']='MPO_3Days_#'+str(updated_header_jsoc[f][recnum].values)
        if mpo_1Week == False and mpo_3Days == False:
            aiamap_list[f].meta['mpo_rec']='MPO_3Days_#'+str(updated_header_jsoc[f][recnum].values)        
    
        #Step 6: updating certain values based on updated JSOC values
        aiamap_list[f].meta['crota2'] = aiamap_list[f].meta['sat_rot'] + updated_header_jsoc[f][instrot_pointing].values #sat_rot + inst_rot, rotation for array axes to get to image axes (deg)
        aiamap_list[f].meta['crpix1'] = updated_header_jsoc[f][x0_pointing].values + 1.
        aiamap_list[f].meta['crpix2'] = updated_header_jsoc[f][y0_pointing].values + 1. 
        aiamap_list[f].meta['cdelt1'] = updated_header_jsoc[f][imscale_pointing].values
        aiamap_list[f].meta['cdelt2'] = updated_header_jsoc[f][imscale_pointing].values
    
        #Step 7: updating xcen and ycen. Equations sourced from comp_fits_cen
        aiamap_list[f].meta['xcen'] = aiamap_list[f].meta['crval1'] + aiamap_list[f].meta['cdelt1']*((float(aiamap_list[f].meta['naxis1'])+1.0)/2.0 - aiamap_list[f].meta['crpix1'])
        aiamap_list[f].meta['ycen'] = aiamap_list[f].meta['crval2'] + aiamap_list[f].meta['cdelt2']*((float(aiamap_list[f].meta['naxis2'])+1.0)/2.0 - aiamap_list[f].meta['crpix2'])    
    #Step 8: Returning the AIAMaps
    bb=[] #trying to force it. doesn't help though
    for g in np.arange(0,n_img):
        map_head=sunpy.map.Map((aiamap_list[g].data, aiamap_list[g].meta))
        bb.append(map_head)
    
    return(bb) #returning the updated aiamap(s)
#----------------------------------------------------------------------------------------------------------------------
def sdo_master_pointing(aiamap,mpo_1Week=False,mpo_3Days=False,cutout=False): #will pass in one AIAMap at a time
    """
    This function accesses the master pointing list to update the header of the level 1 data. The following parameters
    are returned: record number, sun center coordinates, instrument rotation, image scale in arcsec/pixel of CCD.
    This is a generated structure contains reference values of WCS keywords, that are to be used as a 
    reference for aligning an SDO image, either full frame or partial frame
    
    Calling options (Copy-Paste into command line): 
    1. sdo_master_pointing(aiamap,mpo_3Days=True,cutout=False) #Full-disk image(s) provided, 
                                                                   use latest pointing data updated every 3 days
    2. sdo_master_pointing(aiamap,mpo_1Week=True,cutout=False) #Full-disk image(s) provided, 
                                                                    use 1 Week pointing data
    3. sdo_master_pointing(aiamap) #Full-disk image(s) provided, use latest pointing data updated every 3 days
    
    4. sdo_master_pointing(aiamap, mpo_3Days=True, cutout=True)#cutout provided, use latest pointing data updated every 3 days
    5. sdo_master_pointing(aiamap, mpo_1Week=True, cutout=True)#cutout provided, use pointing data updated weekly
    6. sdo_master_pointing(aiamap, cutout=True) #cutout provided, 3 day updated pointing data will be used
    """
    from datetime import datetime, time, date
    print('Accessing Master Pointing Lists')
    jsoc_query_result=[] #contains jsoc master pointing data (returned)
    #--------------------------------------------------------------------------------------------------------------------    
    #Step 1: Ensuring the AIAMap(s) input is a list, when more than one AIAMap is passed it will be a list, but 
             #if only 1 AIAMap is passed, it will have to be converted to a list of 1. 
    aiamap_list=[]
    aiamap_copy=copy.deepcopy(aiamap)
    if isinstance(aiamap_copy,__builtins__.list)!=True: #in order to loop through one or more AIAMaps, they have to be in list form
        aiamap_list.append(aiamap_copy) #if it's not a list, make it a list (only happens when it's just 1 AIAMap)
    if isinstance(aiamap_copy,__builtins__.list)==True:
        aiamap_list=aiamap_copy 
    #--------------------------------------------------------------------------------------------------------------------    
    #Step 2: Checking Inputs
    sig=signature(sdo_master_pointing) #contains the parameters and arguments passed into the function sdo_master_pointing
    params=sig.parameters  #writes out the parameters and their associated values here    
    #print(params) #print(len(params))
    if(len(params) < 1):    #making sure aiamap and jsoc pointing data are passed
        raise ValueError("Not enough parameters passed into the function, kindly provide atleast one AIAMap and the JSOC Pointing Info file")
    if not isinstance(aiamap_list[0], (AIAMap, HMIMap)):
        raise ValueError("Input must be an AIAMap or HMIMap.")
    #-------------------------------------------------------------------------------------------------------------------            
    #Step 3: Looping through each inputed AIAMap
    n_img=np.size(aiamap_list) #List of 1 or more AIAMaps
    print('Number of AIAmaps are {0}'.format(n_img))
    for f in np.arange(0,n_img): #-------------------For each AIAMap-----------------------
        print('For AIAMap # {0}'.format(f+1))    
        #Step 2: Using AIA Map Information used to construct custom JSOC Keyword names
        wavelnth=aiamap_list[f].wavelength #astropy.units.quantity.Quantity, value+units
        wavelnth_num=wavelnth.value #Just the numerical portion of the wavelength
        x0_pointing='A_'+str(int(wavelnth_num))+'_X0'
        y0_pointing='A_'+str(int(wavelnth_num))+'_Y0'
        instrot_pointing='A_'+str(int(wavelnth_num))+'_INSTROT'
        imscale_pointing='A_'+str(int(wavelnth_num))+'_IMSCALE'

        query_key='T_START , '+x0_pointing+' , '+y0_pointing+' , '+instrot_pointing+' , '+imscale_pointing+' , '+'*recnum*'
        t_stamp=aiamap_list[f].date #this is the timestamp of the fits file
        print('The query key sent to JSOC is : '+query_key)
        print('The time of this particular AIAMap is %s'%(t_stamp.time()))
        #--------------------------------------------------------------------------------------------------------------------
        #Step 4: Downloading the data from JSOC if mpo_3Days is set to true:
        if mpo_3Days==False and mpo_1Week==False: #If user doesn't specify preference, set mpo_3Days as true
            mpo_3Days=True 
        
        if mpo_3Days==True:
            c=drms.Client() #all series are available by calling the Client.series() method
            #print(c.series(r'aia')) #this gives all series that begin with 'aia'
            k=c.query('aia_test.master_pointing3h',key=query_key) 
            #---------------------------------------------------------------------------------------------------------------------
            #Find the pointing data at the closest timestamp to the AIAMap
            jsoc_datetime_str=k['T_START'] #the datatime is an object, this becomes a Panda Series

            #Converting the date-time, which is currently in string format to a date-time array:
            i=0
            datetime_dict=[] #empty dictionary which will contain the datetime arrays from JSOC 

            while(i<jsoc_datetime_str.size): #creating date-time array from JSOC data
                first_str=jsoc_datetime_str[i] 
                date_str=first_str[0:10] #just the date
                time_str=first_str[11:19] #just the time
                year_str=int(first_str[0:4]) #forcing it since I am unable to use strptime due to leap sec incapabilities
                month_str=int(first_str[5:7])
                day_str=int(first_str[8:10])
                hour_str=int(first_str[11:13])
                min_str=int(first_str[14:16])
                sec_str=int(first_str[17:19])
    
                if (sec_str > 59):
                    sec_str=59 #forcing the leap second (60) to work
                
                datetime_arr=datetime(year_str,month_str,day_str,hour_str,min_str,sec_str) 
                datetime_dict.append(datetime_arr)
                i+=1

            datetime_pd=pd.Series(datetime_dict) #constructing Panda series

            #finding the difference between the datetime of the AIAMap with the datetime from JSOC
            delta_list=[] #list for appending
            for date in datetime_pd:
                delta=date-t_stamp
                delta1=(abs(delta.total_seconds())) #absolute value of the time difference in seconds
                delta_list.append(delta1)
    
            timediff=pd.Series(delta_list)

            #Which timedifference is minimum?
            index_time_min=timediff.index[timediff==min(timediff)].tolist() #at this index in all the panda Series & DataFrames, the time is closest to the AIAMap time
            b=k.iloc[index_time_min]
            jsoc_query_result.append(b)
        #--------------------------------------------------------------------------------------------------------------------
        #Step 5: Downloading the data from JSOC if mpo_1Week is set to true:
        if mpo_1Week==True:
            c=drms.Client() #all series are available by calling the Client.series() method
            #print(c.series(r'aia')) #this gives all series that begin with 'aia'
            k=c.query('sdo.master_pointing',key=query_key) 
            #---------------------------------------------------------------------------------------------------------------------
            #Find the pointing data at the closest timestamp to the AIAMap
            jsoc_datetime_str=k['T_START'] #the datatime is an object, this becomes a Panda Series

            #Converting the date-time, which is currently in string format to a date-time array:
            i=0
            datetime_dict=[] #empty dictionary which will contain the datetime arrays from JSOC 

            while(i<jsoc_datetime_str.size): #creating date-time array from JSOC data
                first_str=jsoc_datetime_str[i] 
                date_str=first_str[0:10] #just the date
                time_str=first_str[11:19] #just the time
                #datetime_arr=datetime.strptime(first_str,'%Y-%m-%dT%H:%M:%SZ')
                year_str=int(first_str[0:4]) #forcing it since I am unable to use strptime due lack of leap sec capability
                month_str=int(first_str[5:7])
                day_str=int(first_str[8:10])
                hour_str=int(first_str[11:13])
                min_str=int(first_str[14:16])
                sec_str=int(first_str[17:19])
    
                if (sec_str > 59):
                    sec_str=59 #forcing the leap second (60) to work
        
                datetime_arr=datetime(year_str,month_str,day_str,hour_str,min_str,sec_str) 
                datetime_dict.append(datetime_arr)
                i+=1

            datetime_pd=pd.Series(datetime_dict)

            #finding the difference between the datetime of the AIAMap with the datetime from JSOC
            delta_list=[] #list for appending
            for date in datetime_pd:
                delta=date-t_stamp
                delta1=(abs(delta.total_seconds())) #absolute value of the time difference in seconds
                delta_list.append(delta1)
    
            timediff=pd.Series(delta_list)
            #Which timedifference is minimum?
            index_time_min=timediff.index[timediff==min(timediff)].tolist() #at this index in all the panda Series & DataFrames, the time is closest to the AIAMap time
            b=k.iloc[index_time_min]
            jsoc_query_result.append(b)
            

    return(jsoc_query_result) #returns list of jsoc pointing data for each AIAMap
#-----------------------------------------------------------------------------------------------------------------------------------
#executable commands, which would otherwise be written out in the terminal
#aiamap = sunpy.map.Map(sunpy.data.sample.AIA_171_IMAGE) #using a sample data file for code testing purposes
maps = Fido.search(a.Time('2018/3/6 15:00','2018/3/6 15:00:30'),a.Instrument('AIA'),a.Wavelength(171.0*u.Angstrom)) #Fido is used toaccess data
         #This returns an UnifiedResponse object containing information on the available online files which fit the criteria 
         #specified by the attrs objects in the above call. It does not download the files.

aiamap_list1=Fido.fetch(maps) #these files get saved in home dir -> sunpy -> data, or specify path=\'..........\'
aiamap=sunpy.map.Map(aiamap_list1)
print(aiamap[0].meta)
#EXECUTRE THE FOLLOWING SINGLE LINE OF CODE BY ITSELF
maps_new=aia_prep(aiamap,mpo_1Week=True,cutout=False)
#EXECUTE THE FOLLOWING 2 STATEMENTS ONE AFTER THE OTHER
#return_result=sdo_master_pointing(aiamap,mpo_3Days=True)
#aiamaps_return=header_update(aiamap,return_result)  #LIMITATION: Need to access maps one at a time, header and data separately
#-----------------------------------------------------------------------------------------------------------------------------------   

[                                        ========================================
MetaDict([('xtension', 'IMAGE'), ('bitpix', 16), ('naxis', 2), ('naxis1', 4096), ('naxis2', 4096), ('pcount', 0), ('gcount', 1), ('bld_vers', 'V9R1X'), ('lvl_num', 1.0), ('t_rec', '2018-03-06T15:00:10Z'), ('trecstep', 1.0), ('trecepoc', '1977.01.01_00:00:00_TAI'), ('trecroun', 1), ('origin', 'SDO/JSOC-SDP'), ('date', '2018-03-10T07:49:41'), ('telescop', 'SDO/AIA'), ('instrume', 'AIA_3'), ('date-obs', '2018-03-06T15:00:09.35'), ('t_obs', '2018-03-06T15:00:10.35Z'), ('camera', 3), ('img_type', 'LIGHT'), ('exptime', 1.99958), ('expsdev', 0.000157), ('int_time', 2.273438), ('wavelnth', 171), ('waveunit', 'angstrom'), ('wave_str', '171_THIN'), ('fsn', 168104424), ('fid', 0), ('quallev0', 0), ('quality', 0), ('totvals', 16777216), ('datavals', 16777216), ('missvals', 0), ('percentd', 100.0), ('datamin', -5), ('datamax', 3241), ('datamedn', 118), ('datamean', 136.71), ('datarms', 148.26), ('dataskew', 2.08), ('d

/Users/mkocher/anaconda3/lib/python3.7/site-packages/drms/client.py:1089: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res_key = pd.DataFrame.from_items(zip(names, values))


-----------------------------------------
The information that needs to be updated in the meta data for aiamap # 0 are listed below:
[                  T_START     A_171_X0    A_171_Y0  A_171_INSTROT  \
418  2018-03-04T00:00:00Z  2055.399902  2046.01001       0.019327   

     A_171_IMSCALE  *recnum*  
418       0.599489      1114  ]
-----------------------------------------
Header Update Begins Now
Accessing Master Pointing Lists
Number of AIAmaps are 1
For AIAMap # 1
The query key sent to JSOC is : T_START , A_171_X0 , A_171_Y0 , A_171_INSTROT , A_171_IMSCALE , *recnum*
The time of this particular AIAMap is 15:00:21.350000
-----------------------------------------
The information that needs to be updated in the meta data for aiamap # 1 are listed below:
[                  T_START     A_171_X0    A_171_Y0  A_171_INSTROT  \
418  2018-03-04T00:00:00Z  2055.399902  2046.01001       0.019327   

     A_171_IMSCALE  *recnum*  
418       0.599489      1114  ]
-------------------------------

In [244]:
#Notes:
aiamaps_return[0].data #this works
print(aiamaps_return[0].meta) #this works
print(aiamap[0].meta) #it is updating aiamap as well for some reason
#aiamaps_return #does not work, ValueError: object too deep for desired array
#aiamaps_return #does not work

MetaDict([('xtension', 'IMAGE'), ('bitpix', 16), ('naxis', 2), ('naxis1', 4096), ('naxis2', 4096), ('pcount', 0), ('gcount', 1), ('bld_vers', 'V9R1X'), ('lvl_num', 1.5), ('t_rec', '2018-03-06T15:00:10Z'), ('trecstep', 1.0), ('trecepoc', '1977.01.01_00:00:00_TAI'), ('trecroun', 1), ('origin', 'SDO/JSOC-SDP'), ('date', '2018-03-10T07:49:41'), ('telescop', 'SDO/AIA'), ('instrume', 'AIA_3'), ('date-obs', '2018-03-06T15:00:09.35'), ('t_obs', '2018-03-06T15:00:10.35Z'), ('camera', 3), ('img_type', 'LIGHT'), ('exptime', 1.99958), ('expsdev', 0.000157), ('int_time', 2.273438), ('wavelnth', 171), ('waveunit', 'angstrom'), ('wave_str', '171_THIN'), ('fsn', 168104424), ('fid', 0), ('quallev0', 0), ('quality', 0), ('totvals', 16777216), ('datavals', 16777216), ('missvals', 0), ('percentd', 100.0), ('datamin', -5), ('datamax', 3241), ('datamedn', 118), ('datamean', 136.71), ('datarms', 148.26), ('dataskew', 2.08), ('datakurt', 10.6), ('datacent', 148.99), ('datap01', 0.0), ('datap10', 4.0), ('datap